# Rapport de projet de session - IFT 712
### Objectif du projet : tester six méthodes de classification sur une base de données Kaggle.

Nous avons choisi la base de données "[Heart Failure Prediction Dataset][0]" puisqu'elle permet de faire de la classification sur un jeu de données réel et avec des applications concrètes.
Les méthodes de classification que nous allons tester sont les suivantes :
* Réseau de neurones
* K plus proches voisins
* Régression logistique
* Modèle Gaussien naïf
* SVM
* Forêt aléatoire

Pour cela, nous utiliserons la bibliothèque scikit-learn pour implémenter les algotihmes ainsi que pandas  pour traiter les données

Nous utiliserons également [Trello][1] ainsi que discord afin d'organiser le projet à haut niveau
Le code est versionné sur [Github][1] en suivant les conventions suivantes :
* conventionals [commits][3]
* merge requests sur master
* une branche par feature

Le code et les commentaires sont rédigés en francais et suivant la convention [pep8][4] au possible. Nous utiliserons la fonctionnalité "code with me" de pycharm permettant à plussieurs membres du groupe de coder sur le même projet en même temps

[0]: https://www.kaggle.com/datasets/fedesoriano/heart-failure-prediction
[1]: https://trello.com/b/U21MHLaj/projet-ift712-deadline-11-12-23
[2]: https://github.com/MorganChabaudENSSAT/projet_ift712
[3]: https://www.conventionalcommits.org/en/v1.0.0/
[4]: https://peps.python.org/pep-0008/

In [18]:
'''
 Imporation des bibliothèques python générales
'''
import numpy as np
import pandas as pd
from sklearn.exceptions import ConvergenceWarning
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import warnings

'''
 Imporation des bibliothèques spécifiques au devoir
'''
from regression_logistique import RegressionLogistique
from svm import Svm
from reseau_neurones import Reseau_neurones
import utils

'''
    Suppression des Future Warnings 
'''
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=ConvergenceWarning)
warnings.simplefilter(action='ignore')

SyntaxError: invalid syntax (svm.py, line 17)

In [ ]:
# Importation des données
df = pd.read_csv('heart.csv') # Dataframe contenant les données
features_names = df.columns
features_nbr = features_names.shape[0]
print(f"nombre de features dans le dataset : {features_nbr}")
# Visualisation des données pour mieux les comprendre
print(df.head())
print(df.dtypes)

In [ ]:
# A partir de cette visualisation primaire des données, on  remarque que certaines features ne sont pas numériques ce qui empêche de les utiliser telles quelles dans les algorithmes de classification.
# => On va donc devoir traiter ces valeurs en les encodant.
le = LabelEncoder()

data = df.copy(deep = True)

data['Sex'] = le.fit_transform(data['Sex'])
data['ChestPainType'] = le.fit_transform(data['ChestPainType'])
data['RestingECG'] = le.fit_transform(data['RestingECG'])
data['ExerciseAngina'] = le.fit_transform(data['ExerciseAngina'])
data['ST_Slope'] = le.fit_transform(data['ST_Slope'])

# Les données sont mainteant toutes numériques et utilisables par les algorithmes de classification que nous mettrons en place
print(data)

categorical_features = ['Sex','ChestPainType','RestingECG','ExerciseAngina','ST_Slope']
numerical_features = [x for x in features_names if x not in categorical_features]
numerical_features = numerical_features[:-1]

A présent, on explore les données à l'aide de visualisations afin de mieux comprendre leur nature et détecter les features les plus caractéristiques ainsi que la nature des distributions des données afin, au besoin, de formuler des hypothèses nous permettant de mettre en oeuvre différents modèles.
On commence par visualiser la distribution de chaque feature.

In [ ]:
fig = plt.figure(figsize=(10,16))
for i in range(features_nbr):
    plt.subplot(int(features_nbr/2), 2, i+1)
    fig.tight_layout()
    cur_feature =features_names[i]
    sns.distplot(data[cur_feature], kde_kws={'bw' : 1})
    plt.title("Distribution de "+cur_feature)

# Régression logistique #

In [ ]:
# Création d'un modèle de base avec un pénalité l2 et un facteur de prise en compte de la pénalité de 1 centième
reg_log_naif=RegressionLogistique(data, features_names, features_nbr)

On met à l'echelle les données afin de réduire l'impact des différentes valeurs que peuvent prendre les données sur la précision du modèle.
On normalise et standardise toutes les données numériques.

In [ ]:
# Mise à echelle des données
scaled_data = reg_log_naif.scale_data(numerical_features,numerical_features)

On peut alors récupérer les ensembles d'entrainement et de test et procéder à l'entraînement du modèle.

In [ ]:
# Récupération des ensembles d'entraînements et de validation
X_train, X_test, y_train, y_test = reg_log_naif.split_data(scaled_data)
# Entraînement du modèle sur les données d'entraînement
reg_log_naif.train(X_train, y_train)

on commence par évaluer la capacité d'apprentissage du modèle en appliquant un algorithme de cross validation stratifié.

In [ ]:
reg_log_naif.K_fold(X_train,y_train)

Le score de cross-validation du modèle est:  0.848741206960385


On évalue le modèle sur les données de test.

In [ ]:
# Evaluation du modèle sur les données de test
reg_log_naif.evaluate_model(X_test,y_test)

On constate que les score de cross-validation et d'accuracy sont a peu près les même : 85%
Cela laisse penser que le modèle ne sur apprend pas les données.
On vérifie cela rapidmeent en évaluant le modèle sur l'ensemble d'entraînement

In [ ]:
reg_log_naif.evaluate_model(X_train, y_train)

La remarque précédente es validée : il n'y a pas de sur apprentissage.
On va donc cherche maintenant à augmenter la capacité du modèle.

### Recherche d'un modèle de plus grande capacité ###
On cheche a présent à obtenir un modèle de plus grande qualité. Pour cela, on va comparer plusieurs régressions logistiques initialisées avec des hyperparamètres dfférents afin d'obtenir la combinaison optimal des hyperparamètres.
D'après le site de la bibliothèque sklearn, les hyperparamètres sur lesquels il est le plus important d'influer dans le cadre de la régression logistique et que nous étudierons en conséquence sont les suivants:
- le 'solver' c'est à dire l'algorithme d'optimisation qui minimise la loss
- 'penalty' qui correspond à la norme employée dans le terme de régularisation
- 'C' est la constante qui régule l'impact du terme de régularisation (c'est l'inverse de la force de régularisation)

Par la suite, on cherche donc a analyser les résultats des différentes combinaisons d'hyperparamètres de sorte à maximiser la capacité du système **tout en** l'empêchant d'overfitter sur les données. Pour cela, on visualisera l'évolution de l'accuracy sur les ensembles d'entraînement et de validation en procédant à une recherche d'hyperparamètres de type 'grid-search' reposant sur la 'cross_validation'.


In [ ]:
config_1 = {
    'solver' : ['lbfgs', 'liblinear', 'newton-cg', 'sag', 'saga'],
    'penalty' : ['l2'],
    'C' : np.logspace(-4,4,40)
}

config_2 = {
    'solver' : ['lbfgs', 'newton-cg', 'sag', 'saga'],
    'penalty' : ['none'],
    'C' : np.logspace(-4,4,30)
}

config_3 = {
    'solver' : ['liblinear', 'saga'],
    'penalty' : ['l1'],
    'C' : np.logspace(-4,4,30)
}

config_4 = {
    'solver' : ['saga'],
    'penalty' : ['elasticnet'],
    'l1_ratio' : np.linspace(0,1,20),
    'C' : np.logspace(-4,4,30)
}

h_parameters_to_tune =[config_1, config_2, config_3, config_4]


In [ ]:
post_grid_search_estimator = reg_log_naif.hyper_parameters_search(X_train, y_train, h_parameters_to_tune)

Malgré la recherche d'hyperparamètre par grid-search, on trouve un modèle qui présente moins d'accuracy sur les données que le modèle naïf.
Cela peut être dû a une exploration trop peu exhaustive des configurations d'hyperparamètres possibles
On procède tout de même à l'évaluation de ce modèle.

In [ ]:
reg_log_2 = RegressionLogistique(scaled_data, features_names, features_nbr, post_grid_search_estimator)

In [ ]:
reg_log_2.evaluate_model(X_test, y_test)

In [ ]:
reg_log_2.K_fold(X_train,y_train)

#### Amélioration du modèle : Nombre de données

On trace les courbes d'entraînement et de validation en fonction du nombre de données.

In [ ]:
train_size = np.linspace(0.2,1,16)
reg_log_2.data_needed_for_max_score(X_train, y_train, train_size)

On constate que le modèle stagne sur l'entraînement. Cependant, la validation augmente bien avec le nombre de données. Cela semble indiquer que les paramètres choisis pour la régression logistique ne sont pas optimaux.
Or, la recherche d'hyperparamètres permet de trouver les paramètres optimaux donc on peut supposer que les données ne sont pas assez explicite. il faudrait donc retravailler sur les données avec des outils de préprocessing.

#### Conclusion sur le modèle de régression logistique

Nous avons construit un modèle de régression logistique permettant de classifier avec environ 85% de précision les données.
Toutefois, un nombre de données accru ainsi qu'un pré processing des données plus poussé (utilisant par exemple une matrice de correlation permettant de distinguer les features les plus importantes) est une piste d'amélioration pour le modèle.

# 2 - K plus proches voisins

Dans cette partie, on classifie les données à l'aide du classifieur par K plus proches voisins.
Nous évaluerons la valeur de l'hyperparamètre K à l'aide d'une recherche de type 'grid-search' utilisant une 'stratified-cross-validation'

On commence par initialiser un classifieur dit 'naïf' avec l'hyperparamètre K=5 afin d'avoir une base sur laquelle travailler.

In [ ]:
from k_plus_proches_voisins import K_PP_voisins

# Instantiation du modèle avec K =5
knn_model_naif =  K_PP_voisins(data, features_names, features_nbr)

On entraîne le modèle sur les données non traitées

# Génération des ensembles d'entrainement et de test
X_train, X_test, y_train, y_test = knn_model_naif.split_data(data)
# Entraînement du modèle sur les données X_train et y_train
knn_model_naif.train(X_train, y_train)
# Evaluation du modèle
knn_model_naif.evaluate_model(X_test,y_test)

Le modèle a une accuracy de 70%, ce qui semble assez faible. Cela est bien sûr causé par l'utilisation du modèle sur des données non transformées.

On normalise puis standardise les données numériques afin de comparer le gain d'accuracy entre les deux modèles.

In [ ]:
# Normalisation des données numériques
scaled_data_knn = knn_model_naif.scale_data(numerical_features,numerical_features)

On Visualise les données afin de nous assurer que les opérations ont été réalisées avec succès

In [ ]:
print(data)
print(scaled_data_knn)

On peut alors entraîner le modèle de classification par K plus proches voisins sur les données transformées

In [ ]:
# Instantiation du modèle avec K =5
knn_model_naif_2 =  K_PP_voisins(data, features_names, features_nbr)
# Génération des ensembles d'entrainement et de test a partir des données transformées
X_train, X_test, y_train, y_test = knn_model_naif_2.split_data(scaled_data_knn)
# Entraînement du modèle sur les données X_train et y_train
knn_model_naif_2.train(X_train, y_train)

On évalue la capacité du modèle à apprendre les données d'entraînement à l'aide de 'stratified-cross-validation'. Les données sur lesquelles est entrainé le modèle sont séparées en un ensemble d'entraînement et de validation permettant d'évaluer la capacité à généraliser du modèle. On évalue le score sur chaque fold puis on en prend la moyenne afin d'obtenir la meilleure évaluation de l'accuracy possible

In [ ]:
knn_model_naif_2.K_fold(X_train,y_train)

On évalue à présent le modèle sur les données de test.

In [ ]:
# Evaluation du modèle sur les données de test
knn_model_naif_2.evaluate_model(X_test, y_test)

Le modèle arrive à prédire correctement l'existence ou l'absence de maladie cardiaque dans environ 82% des cas sur les données de tests.
On constate une forte augmentation de l'accuracy du modèle par rapport au premier grâce au pré-traitement effectué sur les données.

#### Amélioration du modèle : recherche des hyperparamètres

L'analyse de base du classifieur étant établie, on procède maintenant à une recherche d'hyperparamètres. D'après la documentation de [sklearn][1] les hyperparamètres du modèles sont les suivants :
- n_neighbors : le nombre de voisins nécesaires pour la classification du point courant
- weights : La métrique utilisée pour prédire l'appartenance d'un point à une classe ou à une autre
- algorithm : algorithme utilisé pour le calcul
- leaf_size : nombre de points à partir duquel l'algorithme choisi les voisins en force brute
- p : puissance dans la distance de Minkowski indiquant la nature de la mesure
- metric : la métrique utilisée

On conserve la métrique de base (Minkowski) car le choix de la valeur de p permet de choisir la métrique indirectement.
Le jeu de données étant inférieur à 1000 points, on peut utiliser l'approche par force brute directement car le temps de calcul devrait rester raisonnable. Ainsi, on fixe leaf_size à 1.
L'algorithme utilisé pour le calcul des plus proches voisins est laissé à 'auto'.

Il nous reste donc à calculer les valeurs d'hyperparmètres suivants:
- n_neighbors
- weights
- p

On utilise une recherche d'hyperparamètres de type grid-search. Pour cela, on instancie un dictionnaire contenant les couples (noms du paramètres : valeurs prises par ce paramètre)

[1]: https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier

In [ ]:
n_neighbors_values = np.linspace(1,20,20).astype(int) # valeurs prises par l'hyperparamètre n_neighbors
weights_values = ['uniform', 'distance']    # valeurs prises par l'hyperparamètre weights
p_values = np.linspace(1,10,10).astype(int) # valeurs prises par l'hyperparamètre p
h_parameters_to_tune ={
    'n_neighbors' : n_neighbors_values,
    'weights' : weights_values,
    'p' : p_values
}
other_parameters=['leaf-size=1','metric=minkowski']

In [ ]:
knn_post_grid_search_estimator = knn_model_naif.hyper_parameters_search(X_train, y_train, h_parameters_to_tune, other_parameters)

On constate une amélioration du score de cross-validation d'environ 2.5%

On intancie maintenant un nouveau classifieur avec les hyperparamètres que nous venons de trouver puis nous l'évaluons sur les données de test 

In [ ]:
# instanciation du classifieur unitilisant les hyperparamètres trouvés
knn_2=K_PP_voisins(data, features_names, features_nbr,knn_post_grid_search_estimator)
# Evaluation
knn_2.evaluate_model(X_test, y_test)

L'accuracy du modèle s'est améliorée de presque 5% grâce au choix d'hyperparamètres que nous avons fait.

Afin d'améliorer davantage le modèle, on étudie les interactions entre le modèle et les données

#### Amélioration du modèle : nombre de données

A présent, on étudie le nombre de données nécessaire dans l'ensemble d'entraînement afin de favoriser l'apprentissage du modèle. Pour cela, on va évaluer la performance du modèle sur plusieurs ensembles d'entraînement dont les tailles sont différentes et afficher les résultats obtenus sous la forme de graphiques.
Afin de valider la capacité à généraliser du modèle, nous visualiserons également le score obtenu sur l'ensemble des données de validation.

In [ ]:
train_size = np.linspace(0.2,1,8)
knn_2.data_needed_for_max_score(X_train, y_train, train_size)

Ce graphique indique que la valeur d'accuracy représentée par les courbes d'apprentissage et de validation semble augmenter avec le nombre de données ce qui indique que le modèle n'est pas au bout de son apprentissage. Ainsi, si nous disposions de plus de données, nous pourrions rendre le modèle encore plus performant.

#### Conclusion sur le modèle des K plus proches voisins

Le modèle des K plus proches voisins a permis d'obtenir une accuracy de plus de **86%** sur l'ensemble de test aevc les hyperparamètres suivants : **{'n_neighbors': 17, 'p': 1, 'weights': 'uniform'}** ce qui constitue un gain d'environ 16% par rapport au modèle naïf.
Nous avons observé que **le modèle pourrait gagner en accuracy en bénéficiant de plus de données lors de l'entraînement**.

## 3 - Machine à Vecteurs de Support ##

Pour la classification à l'aide de SVM, nous allons vérifier les performances des différents types de noyaux, afin de trouver le meilleur modèle possible.
Dépendemment du type de noyau utilisé, nous tenterons d'optimiser les hyper-paramètres de chaque modèle afin de trouver le meilleur modèle pour chaque noyau, puis nous comparerons les résultats.
### 3.1 - SVM à noyau linéaire
Dans le cas du noyau linéaire, il n'existe pas d'hyperparamètres à optimiser comme indiqué dans la documentation sklearn de la classe Support Vector Classification.

In [ ]:
# Création du modèle avec noyau linéaire
linear_svm_instance = Svm(data, features_names, features_nbr, model='linear')
# Récupération des ensembles d'entraînements et de validation
X_train, X_test, y_train, y_test = linear_svm_instance.split_data(data)
# Entraînement du modèle sur les données d'entraînement
linear_svm_instance.train(X_train, y_train)
# Evaluation du modèle
linear_svm_instance.evaluate_model(X_test,y_test, confusion_matrix=True)

In [ ]:
train_size = np.linspace(0.2, 1, 8)
linear_svm_instance.plot_learning_curves(X_train, y_train, train_size)

Le fait que les courbes de score évoluent sensiblement de la même façon à partir d'une taille de 350 indique que l'ajustement est relativement approprié.

### 3.1 - SVM à noyau Radial Basis Function
On s'intéresse maintenant à la classification par machines à vecteurs de support avec un noyau RBF. Dans ce cas, les paramètres optimisables sont gamma et C.
Le paramètre C équilibre la classification incorrecte des exemples d'entraînement par rapport à la simplicité de la surface de décision.Plus C est grand, plus le modèle risque le sur-apprentissage. Le paramètre gamma définit l'influence qu'a un exemple d'entraînement unique, et donc plus gamma est grand, plus les autres exemples doivent être proches pour être affectés. Observons d'abord le résultat brut de classification sans optimisation des hyper-paramètres du modèle à noyau RBF.

In [ ]:
# Création du modèle avec noyau rbf
rbf_svm_instance = Svm(data, features_names, features_nbr, model='rbf')
# Récupération des ensembles d'entraînements et de validation
X_train, X_test, y_train, y_test = rbf_svm_instance.split_data(data)
# Entraînement du modèle sur les données d'entraînement
rbf_svm_instance.train(X_train, y_train)
# Evaluation du modèle
rbf_svm_instance.evaluate_model(X_test,y_test)

Observons les scores d'entraînement et de validation avant optimisation.

In [ ]:
rbf_svm_instance.plot_learning_curves(X_train, y_train, train_size)

On va maintenant tenter d'optimiser les paramères gamma et C à l'aide d'un GridSearch de sorte à obtenir le meilleur modèle possible. Ces paramètres sont cruciaux pour la performance des SVM.

In [ ]:
rbf_svm_best_estimator_post_grid = rbf_svm_instance.hyper_parameters_search(X_train, y_train)
best_rbf_svm = Svm(data, features_names, features_nbr, rbf_svm_best_estimator_post_grid)
best_rbf_svm.evaluate_model(X_test, y_test)

In [ ]:
best_rbf_svm.plot_learning_curves(X_train, y_train, train_size)

On remarque que la précision du modèle est la même après optimisation des paramètres. Les courbes de score sont également les mêmes. On aurait pu s'abstenir d'optimiser le paramètre C, comme indiqué dans la documentation sklearn. En effet, il est conseillé de ne pas modifier la valeur de C sauf si les données sont très bruitées.

### 3.3 - SVM à noyau sigmoïde
On s'intéresse maintenant à la classification par machines à vecteurs de support avec un noyau sigmoide. Dans ce cas, le paramètre optimisable est coef0. On observe d'abord le résultat sans optimisation des paramètres.

In [ ]:
# Création du modèle avec noyau sigmoid
sigmoid_svm_instance = Svm(data, features_names, features_nbr, model='sigmoid')
# Récupération des ensembles d'entraînements et de validation
X_train, X_test, y_train, y_test = sigmoid_svm_instance.split_data(data)
# Entraînement du modèle sur les données d'entraînement
sigmoid_svm_instance.train(X_train, y_train)
# Evaluation du modèle
sigmoid_svm_instance.evaluate_model(X_test,y_test)

In [ ]:
sigmoid_svm_instance.plot_learning_curves(X_train, y_train, train_size)

Les deux courbes de score baissent drastiquement. Cela peut indiquer que le modèle n'est pas bien spécifié ou encore que les données sont mal prétraitées. Essayons d'optimiser les hyper-paramètres afin d'améliorer le fonctionnement du modèle.

In [ ]:
sigmoid_svm_best_estimator_post_grid = sigmoid_svm_instance.hyper_parameters_search(X_train, y_train)
best_sigmoid_svm = Svm(data, features_names, features_nbr, sigmoid_svm_best_estimator_post_grid)
best_sigmoid_svm.evaluate_model(X_test, y_test)

In [ ]:
best_sigmoid_svm.plot_learning_curves(X_train, y_train, train_size)

On constate une nette amélioration du modèle de 8%, ainsi que de bien meilleures courbes de score. En effet, les courbes suivent à peu près la même évolution, indiquant que les ajustements sont plutôt corrects.

### 3.4 - SVM à noyau poly
Pour le dernier noyau étudié dans le cadre de ce projet, à savoir le noyau polynomial, celui-ci peut-être optimisé par les hyper-paramètres degree et coef0. Observons les résultats du modèle avant de tenter d'optimiser les paramètres à l'aide d'un GridSearch.

In [ ]:
# Création du modèle avec noyau poly
poly_svm_instance = Svm(data, features_names, features_nbr, model='poly')
# Récupération des ensembles d'entraînements et de validation
X_train, X_test, y_train, y_test = poly_svm_instance.split_data(data)
# Entraînement du modèle sur les données d'entraînement
poly_svm_instance.train(X_train, y_train)
# Evaluation du modèle
poly_svm_instance.evaluate_model(X_test,y_test)

In [ ]:
poly_svm_instance.plot_learning_curves(X_train, y_train, train_size)

Les courbes de score des modèles ne sont pas très bonnes. En effet, l'effet "dent-de-scie" observé sur la courbe du score de validation indique que le modèle sur-apprend. Passons à l'optimisation des hyper-paramètres du modèle.

In [ ]:
poly_svm_best_estimator_post_grid = poly_svm_instance.hyper_parameters_search(X_train, y_train)
best_poly_svm = Svm(data, features_names, features_nbr, poly_svm_best_estimator_post_grid)
best_poly_svm.evaluate_model(X_test, y_test)

In [ ]:
best_poly_svm.plot_learning_curves(X_train, y_train, train_size)

L'amélioration du modèle est faible (1%) après optimisation des hyper-paramètres, et ceci est vérifié par la visualisation des scores qui est sensiblement la même qu'avant optimisation des hyper-paramètres.

### 3.5 - Comparaison des noyaux
Après étude des modèles en fonction des noyaux et optimisation des hyper-paramètres, on remarque que les modèles qui ont la meilleure précision sont les modèles à noyaux rbf et polynomial. Pour autant, la classification par noyau polynomial semble plus prometteuse car les scores d'entraînement et de validation stagnent au fur et à mesure que la taille de l'ensemble d'entraînement augmente.

## 4 - Classification par réseau de neurones
Cette partie est consacrée à l'étude de la classification à l'aide de réseau de neurones. On utilise la classe définie par la bibliothèque sklearn nomée MLPClassifier.
Avant de pouvoir utiliser le modèle, nous devons standardiser et normaliser les données. Nous utiliserons les classes Min-Max-Scaler et StandardScaler de la bibliothèque sklearn.

In [ ]:
# Création d'une instance du modèle
neural_network = Reseau_neurones(data, features_names, features_nbr)
features_to_standardise = ['Age', 'RestingBP', 'Cholesterol', 'MaxHR']
features_to_normalise = features_names
scaled_data = neural_network.scale_data(features_to_normalise=features_to_normalise, features_to_standardise=features_to_standardise)
X_train, X_test, y_train, y_test = neural_network.split_data(scaled_data)
neural_network.train(X_train, y_train)
neural_network.evaluate_model(X_test,y_test)

In [ ]:
train_size = np.linspace(0.2, 1, 8)
neural_network.plot_learning_curves(X_train, y_train, train_size)

On remarque que le modèle voit son score d'entraînement diminuer drastiquement lorsque la taille de l'ensemble d'entraînement augmente, tandis que la courbe qui retrace le score de validation augmente. Le modèle a donc tendance à sur-apprendre quand la taille de l'ensemble d'entraînement croît. On cherche à optimiser les hyper-paramètres, ce que l'on va réaliser à l'aide d'un GridSearch.

In [ ]:
neural_network_best_estimator_post_grid = neural_network.hyper_parameters_search(X_train, y_train)
best_neural_network = Reseau_neurones(scaled_data, features_names, features_nbr, neural_network_best_estimator_post_grid)
best_neural_network.train(X_train,y_train)
best_neural_network.evaluate_model(X_test, y_test)

On constate une amélioration d'environ 3% de la précision du modèle après optimisation des hyper-paramètres.
Observons l'apprentissage du réseau de neurones bien paramétré à l'aide de la méthode partial_fit de la classe MLPClassifier de sklearn.

In [ ]:
best_neural_network.plot_learning_curves(X_train, y_train, train_size)

Il semble qu'à partir d'un échantillon de 450 données, le modèle dont les hyper-paramètres sont les meilleurs a tendance a moins sur-apprendre.